In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
import itertools
import mne
from mne.decoding import CSP
from mne import Epochs
from mne.decoding import SPoC
mne.set_log_level(verbose='warning') #to avoid info at terminal
import pickle
import sys
import os

import tensorflow
import keras
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.optimizers import Adam

from tensorflow.python.keras import backend as K
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet

from scipy import stats
from collections import OrderedDict
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from bayes_opt import BayesianOptimization
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
import gc
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

Using TensorFlow backend.


In [2]:
space_LM = [Real(0, 1, "uniform", name='alpha'),
           Real(0, 1, "uniform", name='l1_ratio')]

space_XGB = [Integer(1, 10, name='max_depth'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Real(10**0, 10**1, "uniform", name="gamma")]

space_NN = [Real(low=1e-4, high=1e-2, prior='log-uniform', name='learning_rate'),
              Integer(low=1, high=3, name='num_dense_layers'),
              Integer(low=1, high=10, prior='uniform', name='num_input_nodes'),
              Integer(low=1, high=10, name='num_dense_nodes'),
              Categorical(categories=['relu', 'tanh'], name='activation')]

In [3]:
def optimize_enet(x,y):

    @use_named_args(space_LM)
    def objective(**params):
        reg=ElasticNet(max_iter=1000, normalize=False)
        reg.set_params(**params)
        cval = cross_val_score(reg, x, y, scoring='r2', cv=3)
        cval[np.where(cval < 0)[0]] = 0

        return -cval.mean()

    res_gp = gp_minimize(objective, space_LM, n_calls=20, random_state=0)
    return res_gp


In [4]:
#### read a real data stream file 
PATH_dat = "C:\\Users\\ICN_admin\\Dropbox (Brain Modulation Lab)\\Shared Lab Folders\\CRCNS\\MOVEMENT DATA\\derivatives\\Int_old_grid\\sub_003_sess_right_run_4.p"

In [7]:
raw_ = np.load(PATH_dat, allow_pickle=True)

In [8]:
raw_.keys()

dict_keys(['vhdr_file', 'resamplingrate', 'BIDS_path', 'projection_grid', 'bv_raw', 'ch_names', 'data_', 'subject', 'run', 'sess', 'sess_right', 'used_channels', 'coord_patient', 'proj_matrix_run', 'fs', 'line_noise', 'seglengths', 'normalization_samples', 'new_num_data_points', 'downsample_idx', 'filter_fun', 'offset_start', 'arr_act_grid_points', 'rf_data_median', 'pf_data_median', 'raw_label_baseline', 'label_baseline_corrected', 'label_baseline_corrected_onoff', 'label', 'label_con_true'])

In [13]:
def append_time_dim(arr, y_, time_stamps):
    """
    apply added time dimension for the data array and label given time_stamps (with downsample_rate=100) in 100ms / need to check with 1375Hz
    """
    time_arr = np.zeros([arr.shape[0]-time_stamps, int(time_stamps*arr.shape[1])])
    for time_idx, time_ in enumerate(np.arange(time_stamps, arr.shape[0])):
        for time_point in range(time_stamps):
            time_arr[time_idx, time_point*arr.shape[1]:(time_point+1)*arr.shape[1]] = arr[time_-time_point,:]
    return time_arr, y_[time_stamps:]

In [22]:
raw_["ch_names"]

['STN_RIGHT_0',
 'STN_RIGHT_1',
 'STN_RIGHT_2',
 'ECOG_RIGHT_0',
 'ECOG_RIGHT_1',
 'ECOG_RIGHT_2',
 'ECOG_RIGHT_3',
 'ECOG_RIGHT_4',
 'ECOG_RIGHT_5',
 'ECOG_RIGHT_6',
 'ECOG_RIGHT_7',
 'MOV_RIGHT',
 'MOV_LEFT']

In [19]:
X = raw_["rf_data_median"][:,0,:] # ch 0 STN_RIGHT_0
y = raw_["label_baseline_corrected"][0,:]

In [20]:
X_, y_ = append_time_dim(X, y, time_stamps=5)

In [23]:
optimizer=optimize_enet(x=X_,y=y_)

C:\Users\ICN_admin\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.135183535179675, tolerance: 0.0014794045058607037
  positive)
C:\Users\ICN_admin\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.472250847507858, tolerance: 0.0017448064728221354
  positive)
C:\Users\ICN_admin\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12.127917521893236, tolerance: 0.002518985599938124
  positive)
C:\Users\ICN_admin\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective h

In [25]:
optimizer

          fun: -0.013070825225774296
    func_vals: array([-0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.01307083, -0.01307083, -0.01307083, -0.01307083, -0.01307083,
       -0.01005173, -0.01307083, -0.01093016, -0.        , -0.01164654])
       models: [GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
                         kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=2, noise='gaussian',
                         normalize_y=True, optimizer='fmin_l_bfgs_b',
                         random_state=209652396), GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
                         kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=2, noise='gaussian',
                         normalize_y=True, optimizer='fmin_l_bfgs

In [24]:
optimizer["x"]

[1.0, 0.0]